In [1]:
##The premise of this project is for the implementation a CNN with VGG-16 as a feature selector 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Create an ImageGenerator object that is used to randomize and make certain small transformations to the image
#to build better and robust networks

from keras.preprocessing.image import ImageDataGenerator

In [3]:
image_gen = ImageDataGenerator(rotation_range=30,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              rescale=1/255,
                              zoom_range=0.2,
                              shear_range=0.2,
                              fill_mode='nearest')

## Model:


In [4]:
from keras.applications import vgg16
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras import optimizers

In [5]:
model = vgg16.VGG16(weights='imagenet', include_top=False, 
                    input_shape=(150,150,3), pooling="max")

for layer in model.layers[:-5]:
        layer.trainable = False

for layer in model.layers:
    print(layer, layer.trainable)


transfer_model = Sequential()
for layer in model.layers:
    transfer_model.add(layer)
transfer_model.add(Dense(128, activation="relu"))  
transfer_model.add(Dropout(0.5))
transfer_model.add(Dense(10, activation="softmax")) 

adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.00001)

transfer_model.compile(loss="categorical_crossentropy",
                      optimizer=adam,
                      metrics=["accuracy"])

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f406ea55da0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f406ea00438> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f406ea00fd0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f406c175748> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f406c17d7f0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f406c17d080> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f406c185dd8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f406c18ba90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f406c18fa20> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f406c18fe80> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f406c19a160> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

In [6]:
transfer_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 37, 37, 256)       2

In [7]:
train_directory = '/home/amoghavarsha/Project_CNN/my_project_env/Dataset/training/training'
validation_directory = '/home/amoghavarsha/Project_CNN/my_project_env/Dataset/validation/validation'

In [8]:
## Getting the training and the validation sets
batch_size = 16
train_gen = image_gen.flow_from_directory(train_directory,target_size=(150,150),batch_size=batch_size,
                                         class_mode='categorical')

Found 1098 images belonging to 10 classes.


In [9]:
validation_gen = image_gen.flow_from_directory(validation_directory,target_size=(150,150),batch_size=batch_size,
                                         class_mode='categorical')

Found 272 images belonging to 10 classes.


In [10]:
results = transfer_model.fit_generator(train_gen,epochs=30,steps_per_epoch=1097//batch_size,
                              validation_data=validation_gen,validation_steps=272//batch_size)

/home/amoghavarsha/Project_CNN/my_project_env/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
68/68 [==============================] - 253s 4s/step - loss: 2.5558 - accuracy: 0.1123 - val_loss: 1.9113 - val_accuracy: 0.4338
Epoch 2/30
68/68 [==============================] - 252s 4s/step - loss: 1.8870 - accuracy: 0.3353 - val_loss: 1.4465 - val_accuracy: 0.5331
Epoch 3/30
68/68 [==============================] - 256s 4s/step - loss: 1.4549 - accuracy: 0.4746 - val_loss: 0.8776 - val_accuracy: 0.6985
Epoch 4/30
68/68 [==============================] - 250s 4s/step - loss: 0.8281 - accuracy: 0.7227 - val_loss: 0.7860 - val_accuracy: 0.7574
Epoch 5/30
68/68 [==============================] - 250s 4s/step - loss: 0.5977 - accuracy: 0.8173 - val_loss: 0.9970 - val_accuracy: 0.6691
Epoch 6/30
68/68 [==============================] - 245s 4s/step - loss: 0.5871 - accuracy: 0.8230 - val_loss: 0.7683 - val_accuracy: 0.7757
Epoch 7/30
68/68 [==============================] - 245s 4s/step - loss: 0.3718 - accuracy: 0.8865 - val_loss: 0.7216 - val_accuracy: 0.7941
Epoch 8/30
68

In [11]:
transfer_model.save('TL_CNN_Monkey.h5')

In [23]:
_, acc = transfer_model.evaluate_generator(validation_gen, steps=272 //batch_size)

/home/amoghavarsha/Project_CNN/my_project_env/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [24]:
print('The testing accuracy for the CNN with the 10-Species-Monkey dataset is : %.3f' % (acc * 100.0))

The testing accuracy for the CNN with the 10-Species-Monkey dataset is : 84.559


In [ ]:
from tensorflow import keras
x = keras.models.load_model('TL_CNN_Monkey.h5')